<a href="https://colab.research.google.com/github/rozapkk13/unet/blob/master/trainUnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"  # Force TensorFlow Keras compatibility

import tensorflow as tf
from segmentation_models import Unet  # ✅ Import pre-trained U-Net
from tensorflow.keras.callbacks import ModelCheckpoint


Segmentation Models: using `tf.keras` framework.


In [4]:
import segmentation_models
print("Segmentation Models installed successfully!")


Segmentation Models installed successfully!


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [30]:
import tensorflow as tf
import numpy as np  # ✅ Fix NameError: np is not defined

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy

# ✅ Enable Mixed Precision & XLA
set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

# ✅ Enable GPU Memory Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# ✅ Data Generator Optimization
AUTOTUNE = tf.data.AUTOTUNE

# ✅ Import trainGenerator if it's in another script
# from data_loader import trainGenerator
def convert_grayscale_to_rgb(generator):
    for batch in generator:
        image, mask = batch
        image = np.repeat(image, 3, axis=-1)  # ✅ Convert grayscale to RGB
        yield image, mask
# ✅ Define it here if missing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Define Data Augmentation Arguments
data_gen_args = dict(
    rotation_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ✅ Fix trainGenerator indentation
def trainGenerator(batch_size, train_path, image_folder, label_folder, aug_dict, save_to_dir=None):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)

    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode='grayscale',
        target_size=(256, 256),
        batch_size=batch_size,
        save_to_dir=save_to_dir
    )

    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[label_folder],
        class_mode=None,
        color_mode='grayscale',
        target_size=(256, 256),
        batch_size=batch_size,
        save_to_dir=save_to_dir
    )

    train_generator = zip(image_generator, mask_generator)
    return train_generator

# ✅ Now it should work without indentation errors
myGene = trainGenerator(16, 'data/membrane/train', 'image', 'label', data_gen_args, save_to_dir=None)


myGene = convert_grayscale_to_rgb(myGene)


# ✅ Initialize U-Net Model (Multi-GPU)
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = Unet('resnet34', encoder_weights='imagenet', input_shape=(256, 256, 3))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

print(model.summary())

# ✅ Train the model
model_checkpoint = ModelCheckpoint('unet_membrane.keras', monitor='loss', verbose=1, save_best_only=True)
steps_per_epoch = 500
model.fit(myGene, steps_per_epoch=steps_per_epoch, epochs=5, callbacks=[model_checkpoint])


Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ data (InputLayer)         │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_8 (Cast)             │ (None, 256, 256, 3)    │              0 │ data[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_data                   │ (None, 256, 256, 3)    │              9 │ cast_8[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_136        │ (None, 262, 262, 3)    │              0 │ bn_data[0][0]          │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv0 (Conv2D)            │ (None, 128, 128, 64)   │          9,408 │ zero_padding2d_136[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn0 (BatchNormalization)  │ (None, 128, 128, 64)   │            256 │ conv0[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ relu0 (Activation)        │ (None, 128, 128, 64)   │              0 │ bn0[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_137        │ (None, 130, 130, 64)   │              0 │ relu0[0][0]            │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pooling0 (MaxPooling2D)   │ (None, 64, 64, 64)     │              0 │ zero_padding2d_137[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stage1_unit1_bn1          │ (None, 64, 64, 64)     │            256 │ pooling0[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stage1_unit1_relu1        │ (None, 64, 64, 64)     │              0 │ stage1_unit1_bn1[0][0] │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_138        │ (None, 66, 66, 64)     │              0 │ stage1_unit1_relu1[0]… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stage1_unit1_conv1        │ (None, 64, 64, 64)     │         36,864 │ zero_padding2d_138[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stage1_unit1_bn2          │ (None, 64, 64, 64)     │            256 │ stage1_unit1_conv1[0]… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stage1_unit1_relu2        │ (None, 64, 64, 64)     │              0 │ stage1_unit1_bn2[0][0] │
│ (Activation)              │                        │                │                        │
├──────────────────────

 Total params: 24,456,154 (93.29 MB)

 Trainable params: 24,438,804 (93.23 MB)

 Non-trainable params: 17,350 (67.77 KB)

None
Epoch 1/5
495/500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 1: loss improved from inf to 0.00000, saving model to unet_membrane.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 2/5
497/500 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 2: loss did not improve from 0.00000
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3: loss did not improve from 0.00000
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 4/5
499/500 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 4: loss did not improve from 0.00000
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5
494/500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - 

### Train with npy file

In [31]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [5]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
from segmentation_models import Unet
!pip install segmentation-models efficientnet image-classifiers


from tensorflow.keras.models import load_model



def testGenerator(test_path, target_size=(256, 256), as_gray=True):
    for file_name in os.listdir(test_path):
        img = io.imread(os.path.join(test_path, file_name), as_gray=as_gray)
        img = trans.resize(img, target_size)
        img = np.expand_dims(np.reshape(img, img.shape + (1,)), axis=0)
        yield img

def saveResult(save_path, results):
    os.makedirs(save_path, exist_ok=True)
    for i, item in enumerate(results):
        io.imsave(os.path.join(save_path, f"result_{i}.png"), item[:, :, 0])

testGene = testGenerator("data/membrane/test")
model = Unet()
model = load_model("unet_membrane.keras")  # ✅ Load full model, not just weights# ✅ Change .hdf5 to .keras

results = model.predict(testGene, steps=30, verbose=1)
saveResult("data/membrane/test", results)


58889256/58889256 [==============================] - 0s 0us/step


OSError: No file or directory found at unet_membrane.keras

In [6]:
import os
print(os.getcwd())  # Check current working directory
print(os.listdir())  # List all files in the directory


/content
['.config', 'sample_data']
